#### **Library imports**

In [5]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

#### **Hyperparams and loading data**

In [2]:
train_edges = np.load('data/train_edges.npy')
users = torch.LongTensor(train_edges[:, 0])
items = torch.LongTensor(train_edges[:, 1])
ratings = torch.FloatTensor(train_edges[:, 2])

n_users = 943 
n_items = 1682
n_samples = len(ratings)

#### **Defining collaborative filtering**

In [3]:
class CollaborativeFiltering(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CollaborativeFiltering, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        dot = (u * i).sum(1)
        return torch.sigmoid(dot)

#### **Code for meta attack**

In [12]:
# start execution
start_time = time.time()

# some hyperparams
lr = 1
T = 10
Delta = 20
n_factors = 64

print('hyperparams used --')
print('learning rate: ', lr)
print('inner loop count T: ', T)
print('outer loop count Delta: ', Delta)
print('embedding size: ', n_factors)

# list of perturbations
edges_to_add = []

for delta in range(Delta):
    # reload the users, items and ratings tensors
    users = torch.LongTensor(train_edges[:, 0])
    items = torch.LongTensor(train_edges[:, 1])
    ratings = torch.FloatTensor(train_edges[:, 2])

    # add those perturbations to "ratings"
    for index in edges_to_add:
        ratings[index] = 1
    print('{} edges perturbed in "rating"'.format(len(edges_to_add)))

    # set requires_grad for ratings, to compute meta gradients
    ratings.requires_grad_()

    # makes code reproducible
    torch.manual_seed(0)

    # define model and loss
    model = CollaborativeFiltering(n_users, n_items, n_factors)
    p1, p2 = model.parameters()
    loss_fn = nn.BCELoss(reduction = 'mean')
    model.train()

    print('perturbation #', delta)
    # inner loop training process
    for i in range(T):
        y_hat = model(users, items)
        loss = loss_fn(y_hat, ratings)
        # if i % 5 == 0:
        print('inner loss at iter {}: {}'.format(i, loss.item()))
        
        p1_grad = torch.autograd.grad(loss, p1, create_graph=True)
        p2_grad = torch.autograd.grad(loss, p2, create_graph=True)

        with torch.no_grad():
            p1_new = p1 - lr * p1_grad[0]
            p2_new = p2 - lr * p2_grad[0]

            p1.copy_(p1_new)
            p2.copy_(p2_new)
    
    # compute meta gradient
    meta_grad = torch.autograd.grad(loss, ratings)[0]
    print('meta gradients: ', meta_grad)

    # select best edge to perturb
    max_meta_grad = -math.inf
    edge_to_add = -1
    search_space = 0
    for i in range(n_samples):
        if ratings[i] == 0: # search over only negative edges
            if meta_grad[i] > max_meta_grad:
                max_meta_grad = meta_grad[i]
                edge_to_add = i 
            search_space += 1
    print('edges searched: {}'.format(search_space))
    print('max meta gradient: {}, edge to add: {}'.format(max_meta_grad, edge_to_add))
    edges_to_add.append(edge_to_add)

    print("\n\n")

# finish execution
print('---- Execution time: {} seconds ----'.format(round(time.time() - start_time, 2)))

hyperparams used --
learning rate:  1
inner loop count T:  10
outer loop count Delta:  20
embedding size:  64
0 edges perturbed in "rating"
perturbation # 0
inner loss at iter 0: 813319.1875
inner loss at iter 1: 12675684.0
inner loss at iter 2: 11165178.0
inner loss at iter 3: 10464424.0
inner loss at iter 4: 10077103.0
inner loss at iter 5: 9829036.0
inner loss at iter 6: 9676045.0
inner loss at iter 7: 9586101.0
inner loss at iter 8: 9532800.0
inner loss at iter 9: 9483006.0
meta gradients:  tensor([-inf, -inf, inf,  ..., inf, inf, -inf])
edges searched: 100000
max meta gradient: inf, edge to add: 2



1 edges perturbed in "rating"
perturbation # 1
inner loss at iter 0: 813316.9375
inner loss at iter 1: 12675580.0
inner loss at iter 2: 11164383.0
inner loss at iter 3: 10463269.0
inner loss at iter 4: 10067668.0
inner loss at iter 5: 9818084.0
inner loss at iter 6: 9658024.0
inner loss at iter 7: 9562782.0
inner loss at iter 8: 9516551.0
inner loss at iter 9: 9482342.0
meta gradients